Задание 2.1 (Фильм)

In [1]:
import numpy as np
import pandas as pd
import math

movieRating = pd.DataFrame(pd.read_csv('./data.csv'))

contextPlace = pd.DataFrame(pd.read_csv('./context_place.csv'))
contextDay = pd.DataFrame(pd.read_csv('./context_day.csv'))

def sim(usrIndexA, usrIndexB):
    trgtsRatings = movieRating.iloc[[usrIndexA, usrIndexB], range(1, len(movieRating.columns))]
    commonRatings = trgtsRatings[trgtsRatings != -1].dropna(axis=1)

    usrA = commonRatings.iloc[0, :]
    usrB = commonRatings.iloc[1, :]

    sumUsrA = usrA.apply(lambda x: x * x)
    sumUsrB = usrB.apply(lambda x: x * x)

    sumUsrA = math.sqrt(sumUsrA.sum())
    sumUsrB = math.sqrt(sumUsrB.sum())

    res = (usrA * usrB).sum() / (sumUsrA * sumUsrB)
    return res



def getMean(userID):
    temp = movieRating.iloc[[userID], range(1, len(movieRating.columns))]
    res = temp[temp != -1].dropna(axis=1).mean(axis=1)
    return res.values[0]


def getFilmRating(userID, filmID):
    rating = movieRating.iloc[[userID], range(1, len(movieRating.columns))]
    return rating.iloc[[0], [filmID]].values

def getSimMatrix():
    res = pd.DataFrame(np.zeros(shape=(movieRating.count()[0], movieRating.count()[0])),
                       columns=range(0, movieRating.count()[0]))
    for usrA in range(0, movieRating.count()[0]):
        for usrB in range(0, movieRating.count()[0]):
            if (usrA != usrB):
                res.iat[usrA, usrB] = sim(usrA, usrB)
    return res


def getSimMatrixUser(usrA):
    res = pd.DataFrame(np.zeros(shape=(1, movieRating.count()[0])), columns=range(0, movieRating.count()[0]))

    for usrB in range(0, movieRating.count()[0]):
        if (usrA != usrB):
            res.iat[0, usrB] = sim(usrA, usrB)
    return res


def getEval(userID, filmID):
    userID = userID - 1

    usersK = simMatrix.iloc[0,:].sort_values( ascending=False)

    badColumns = []
    for trgtId in range(0, movieRating.count()[0]):
        if (getFilmRating(trgtId, filmID) == -1 and trgtId != userID):
            badColumns.append(trgtId)

    usersK = usersK[:4]
    usersK = usersK.drop(badColumns, errors='ignore')

    sumSim = np.abs(usersK.sum())
    usersSim = usersK.to_dict();

    res = 0
    for trgtID, sim in usersSim.items():
        rvi = getFilmRating(trgtID, filmID)
        rv = getMean(trgtID);
        res = res + sim * (rvi - rv)
    eval = getMean(userID) + (res / sumSim)
    return [round(eval.item(0), 3), usersK]



res = {}
userID = 26
bestFilm = {}

simMatrix = getSimMatrixUser(userID - 1)



simMatrix

for i in range(0, len(movieRating.columns) - 1):
    rvi = getFilmRating(userID - 1, i)
    maxK = 0
    if (rvi == -1):
        evalRes = getEval(userID, i)
        res['Movie ' + str(i + 1)] = evalRes[0]
        if (len(bestFilm) == 0):
            maxK = len(evalRes[1])
            bestFilm.clear()
            bestFilm['Movie ' + str(i + 1)] = evalRes[0]
        if (len(evalRes[1]) >= maxK and bestFilm[next(iter(bestFilm))] < evalRes[0]):
            maxK = evalRes[1]
            bestFilm.clear()
            bestFilm['Movie ' + str(i + 1)] = evalRes[0]


resJson = {
    'user': userID,
    '1': res,
    '2': bestFilm
}
print(resJson)

{'user': 26, '1': {'Movie 3': 2.456, 'Movie 5': 3.289, 'Movie 8': 2.775, 'Movie 26': 3.554, 'Movie 30': 2.178}, '2': {'Movie 26': 3.554}}


Задание 3.2

22 список имен детей всех актеров фильма с указанием имени актера

In [2]:
bestFilmNumber = next(iter(bestFilm.keys()))

In [3]:
movieNames = pd.DataFrame(pd.read_csv('./movie_names.csv',encoding='ISO-8859-1', header = None))

In [4]:
movieName = movieNames.loc[movieNames[0]==bestFilmNumber][1].values[0]

Делаем запрос на api

In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

In [7]:
api = "https://www.wikidata.org/w/api.php"

params = {
'action' : 'wbsearchentities',
'format' : 'json',
'language' : 'en',
'search': movieName
}

res = requests.get(api, params = params)
res.json()['search'][0]['description']

'1995 German-American neo-noir crime thriller film directed by Bryan Singer'

In [8]:
movieCode = res.json()['search'][0]['id']

In [9]:
movieCode

'Q132351'

In [14]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql_query = """
SELECT distinct ?actorLabel ?child ?childLabel
WHERE {
  wd:"""+movieCode+""" wdt:P161 ?actor.
  ?actor wdt:P40 ?child.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}
 
"""

sparql.setQuery(sparql_query)

sparql.setReturnFormat(JSON)
queryResult = sparql.query().convert()

In [15]:
queryResult

{'head': {'vars': ['actorLabel', 'child', 'childLabel']},
 'results': {'bindings': [{'child': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q20983703'},
    'actorLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Stephen Baldwin'},
    'childLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Hailey Baldwin'}},
   {'child': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q57983542'},
    'actorLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Clark Gregg'},
    'childLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Stella Gregg'}}]}}

In [16]:
gatheredData = pd.json_normalize(results['results']['bindings'])

In [17]:
gatheredData

,child.type,child.value,actorLabel.xml:lang,actorLabel.type,actorLabel.value,childLabel.xml:lang,childLabel.type,childLabel.value
0,uri,http://www.wikidata.org/entity/Q20983703,en,literal,Stephen Baldwin,en,literal,Hailey Baldwin
1,uri,http://www.wikidata.org/entity/Q57983542,en,literal,Clark Gregg,en,literal,Stella Gregg


In [19]:

gatheredData[['actorLabel.value', 'child.value','childLabel.value']]

,actorLabel.value,child.value,childLabel.value
0,Stephen Baldwin,http://www.wikidata.org/entity/Q20983703,Hailey Baldwin
1,Clark Gregg,http://www.wikidata.org/entity/Q57983542,Stella Gregg
